# ATAC-seq Module 2: Visualization and Peak Identification

<img src=../images/Tutorial2/LessonImages/ATACseqWorkflowLesson2.jpg alt="Drawing" style="width: 1000px;"/>

## Overview
In the previous section of this module we performed preprocessing quality control, mapping, and deduplication. In this section we will focus on visualization of the signal, create average plots of signal around transcription start sites (TSSs), and identification of peak signal. 

## Learning Objectives

* **Setting up the computational environment:** Installing necessary bioinformatics tools (samtools, deeptools, macs2, IGV) using mamba and pip.

* **Data organization and management:** Creating a directory structure and copying example ATAC-seq data files (bam files and genome annotations).

* **Data visualization:**
    * Understanding the BAM file format and its contents.
    * Generating bigwig files from BAM files using `bamCoverage` (deeptools) for visualization of read density across the genome.
    * Understanding the concept of BPM (Bins Per Million) normalization.
    * Visualizing the bigwig files using IGV to explore read density across genomic regions.
    * Generating average profiles of read density around Transcription Start Sites (TSSs) using `computeMatrix` and `plotProfile` (deeptools).  This involves understanding the use of BED files for genomic annotations.

* **Peak calling:**
    * Understanding the concept of peaks in ATAC-seq data.
    * Performing peak calling using `macs2` on the ATAC-seq data.
    * Understanding the output of `macs2` (narrowPeak, xls, bed files).
    * Visualizing identified peaks alongside the signal in IGV.

* **Understanding insert size distributions in paired-end ATAC-seq:**
    * Interpreting the distribution of insert sizes to identify nucleosomal fragments and transposase hypersensitive sites (THSS).
    * Using `bamPEFragmentSize` (deeptools) to generate a histogram of insert sizes.

* **Filtering reads by insert size (optional):** Separating reads into nucleosomal and THSS fragments based on insert size using `samtools`.

* **Accounting for Tn5 insertion offset (optional):** Shifting reads using `alignmentSieve` (deeptools) to correct for the 9 bp gap introduced by Tn5.

## Prerequisites

**Software Prerequisites:**

* **Mamba:** Used for managing the software environment and installing packages.
* **samtools:** For manipulating BAM files (indexing and viewing).
* **deeptools:** For generating bigwig files (`bamCoverage`), computing matrix and plotting profiles (`computeMatrix`, `plotProfile`), and analyzing insert sizes (`bamPEFragmentSize`, `alignmentSieve`).
* **macs2:** For peak calling.
* **IGV (Integrative Genomics Viewer):** For visualizing genomic data (bigwig files and peaks).  The notebook uses the `igv-notebook` package for Jupyter integration.
* **jupyterquiz:** For interactive quiz questions within the notebook.
* **pandas:** For data manipulation (though usage is minimal in this example).


**APIs:**

* **Google Cloud Storage (GCS) API:**  The notebook uses `gsutil` to copy example data files from a GCS bucket.  This requires authentication with Google Cloud.

## Get Started
### Required Files
In this stage of the module you will use the deduplicated bam files that we prepared in the previous section. Don't worry if you are just jumping in now, we have examples of these files saved and will include a step that copies them for your use. You can also use this module on your own data or any published ATAC-seq dataset, but you should complete the mapping and deduplication steps first.


### STEP 1: Set Up Environment


Initial items to configure your Google Cloud environment. In this step we will use conda to install the following packages:

Visualization:
[samtools](https://anaconda.org/bioconda/samtools), [deeptools](https://anaconda.org/bioconda/deeptools), [IGV](https://anaconda.org/bioconda/igv)

Peak Identification:
[macs2](https://anaconda.org/bioconda/macs2)

In [ ]:
! mamba install -c conda-forge ncurses -y
! mamba install -c bioconda samtools deeptools macs2 -y
! pip install jupyterquiz
! pip install igv-notebook

In [ ]:
#import igv_notebook
import igv_notebook
import sys
sys.path.insert(0, "/home/jupyter/.local/lib/python3.7/site-packages")
from jupyterquiz import display_quiz
from IPython.display import IFrame
from IPython.display import display
from IPython.display import Image
import pandas as pd
#define the number of threads for parallel processes
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
numthreadsint = int(numthreads[0])

## Set Up File System
Now lets create some folders to stay organized and copy over our prepared fastq files. We're going to work in a directory called "Tutorial2" during this module. We'll then create sub-folders for the files that we'll be creating. 

In [ ]:
# These commands create our directory structure.
! mkdir -p Tutorial2/InputFiles
! mkdir -p Tutorial2/GenomeAnnotations
! mkdir -p Tutorial2/BigWigFiles
! mkdir -p Tutorial2/Peaks
! mkdir -p Tutorial2/Plots

# These commands help identify the google cloud storage bucket where the example files are held.
original_bucket = "gs://nigms-sandbox/unmc_atac_data_examples/Tutorial2"

# This command copies our example files to the Tutorial2 folders that we created above.
! gsutil -m cp $original_bucket/InputFiles/* Tutorial2/InputFiles
! gsutil -m cp $original_bucket/Annotations/* Tutorial2/GenomeAnnotations

### OK
Let's make sure that the files are present. You should see two .bam files after running the following command:

In [ ]:
! ls Tutorial2/InputFiles

<div class="alert-info" style="font-size:200%">
STEP 2: Visualization
</div>
Files in sam/bam format contain a lot of information including the original sequence of the reads, quality scores, and their corresponding chromosomal coordinates.

<img src="../images/Tutorial2/LessonImages/samformat.jpg" alt="Drawing" style="width: 1000px;"/>


### Please view this [site](https://www.samformat.info/sam-format-flag) for a more complete description of the sam format and to see what the various sam flag values mean.

Let's view the first few lines of one of our bam files:

In [ ]:
! samtools view Tutorial2/InputFiles/CTL_dedup.bam | head -3 | cat
# Note that there will be an error message because we are breaking a pipe by printing only the first 3 lines. Please ignore the error message.

While we can see the coordinates of each read, we will need a better way of visualizing the results. In this step we will create a binary file that summarizes the pileup of reads at each base-pair along our genome, in [bigwig](http://genome.ucsc.edu/goldenPath/help/bigWig.html) format. 

To create the bigwig files let's use the command bamCoverage, part of the [deeptools](https://deeptools.readthedocs.io/en/develop/) package.

In [ ]:
# First we need to create an index of our bam file.
! samtools index Tutorial2/InputFiles/CTL_dedup.bam

# Then we can create a bigwig file of the control sample.
! bamCoverage -b Tutorial2/InputFiles/CTL_dedup.bam -o Tutorial2/BigWigFiles/Control.bw -bs 1 -p $numthreadsint --normalizeUsing BPM 2> Tutorial2/BigWigFiles/bamCovLog_ctl.txt

# Now let's rerun the commands for our mutant sample.
! samtools index Tutorial2/InputFiles/Mutant_dedup.bam
! bamCoverage -b Tutorial2/InputFiles/Mutant_dedup.bam -o Tutorial2/BigWigFiles/Mutant.bw -bs 1 -p $numthreadsint --normalizeUsing BPM 2> Tutorial2/BigWigFiles/bamCovLog_mut.txt

print("done")

In the above example we specify the bam file name after -b and the output file name after -o. 

We specified -bs 1, which tells bamCoverage to summarize the reads at every basepair; the default is to summarize at 50 bp resolution, but for ATAC-seq we find it useful to summarize the data at finer-scale. 

We also specified the number of threads to use with -p, which is held in a variable in our notebook.

Lastly, we specified --normalizeUsing BPM. BPM stands for Bins Per Million mapped reads. What do you think this normalization does?

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 1: Click on the correct answer in following cell.
</div>

In [ ]:
display_quiz("../quiz_files/BPMnorm.json")

<div class="alert-info" style="font-size:120%">
Genome Browser
</div>

Now that we have our bigwig files, we can visualize the signal in a genome browser. We'll use [igv](https://igv.org/) in this example.

In [ ]:
igv_notebook.init()
myigv = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr4:55,400,000-56,400,000"
    }
)
myigv.load_track(
{
        "name": "CTL",
        "url": "Tutorial2/BigWigFiles/Control.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "Mutant",
        "url": "Tutorial2/BigWigFiles/Mutant.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)

This will load in the signal into IGV and allow you to browse the genome. Feel free to play around with this. More instructions can be found on the [IGV](https://igv.org/) website. 

Notice that when we first load in the files, the scales are different on the left-hand side. IGV defaults to autoscale each individually. However, if we want to compare to signals we should use the same y-axis scale for both. We can do this because we included BPM normalization. To change the scale, click on the gear icon on the right of each track and select "Set data range". Let's set the maximum to 300 both both. 

<img src="../images/Tutorial2/LessonImages/igv.jpg" alt="Drawing" style="width: 1200px;"/>


In addition to scrolling along the genome, go ahead an try to zoom in on a specific "peak" of signal. You can do so by clicking on the top ruler (where the coordinates are displayed), holding, and dragging either direction. Alternatively, you can click on the + and - signs at the top right.

<div class="alert-info" style="font-size:120%">
Average Profiles
</div>

In addition to browsing, we can make average profiles of signal across specific regions. For example, ATAC-seq signal should be enriched near TSSs. Let's test this using [deeptools](https://anaconda.org/bioconda/deeptools).

Deeptools takes in a bigwig file representing the signal. It also takes a bed file representing the features across which one wants to average the signal. In our case the bed file will be composed of gene annotations. Creating the profile will occur in two steps. The first is to create the summarized matrix, while the second plots that data.

In [ ]:
# -S option specifies the bigwig signal file, where we can specify multiple separated by spaces. -R option specifies the genome annotation bed file. -a and -b specify how many bp to plot on either side. 
! computeMatrix reference-point --referencePoint TSS -S Tutorial2/BigWigFiles/Control.bw Tutorial2/BigWigFiles/Mutant.bw -R Tutorial2/GenomeAnnotations/hg38_genes_chr4.bed -o Tutorial2/Plots/TSSprofileMatrix -a 10000 -b 10000
! plotProfile -m Tutorial2/Plots/TSSprofileMatrix -o Tutorial2/Plots/TSSprofile.png 

Let's view the output:

In [ ]:
Image(url= "Tutorial2/Plots/TSSprofile.png", width=400, height=400)

<div class="alert-info" style="font-size:120%">
A note on insert sizes
</div>

As reported in the original ATAC-seq publication, high quality ATAC-seq datasets reveal a specific distribution of insert sizes that correspond to distinct chromatin features. 
To view an example of this distribution, see the following publication: Buenrostro et al., Transposition of native chromatin for fast and sensitive epigenomic profiling of open chromatin, DNA-binding proteins and nucleosome position, Nat. Methods, 2013.

<img src="../images/Tutorial2/LessonImages/sizeProfile.jpg" alt="Drawing" style="width: 400px;"/>

Here we see abundant insert sizes corresponding to accessible chromatin vs nucleosomal fragments. Note that we only know the insert size with paired-end data, and not with single-end sequencing.

### Nucleosomes consist of 145 bp of DNA wrapped around histones. Because Tn5 randomly inserts near protected sites, in paired-end ATAC-seq this results in a slightly larger range of protected fragments (i.e. insertion sizes). Based on this information, look at the graph and think about the size range that would be most consistent with TF binding vs mono-nucleosomes. 

We can use Deeptools to summarize our insert sizes. 

In [ ]:
! bamPEFragmentSize -b Tutorial2/InputFiles/CTL_dedup.bam Tutorial2/InputFiles/Mutant_dedup.bam -o Tutorial2/Plots/Insertsizes_histogram.png -p $numthreadsint --maxFragmentLength 1000 > Tutorial2/Plots/insertsize_log.txt
print("done")


In [ ]:
Image(url= "Tutorial2/Plots/Insertsizes_histogram.png", width=400, height=400)

<div class="alert-info" style="font-size:200%">
Interactive Quiz Question 2: Click on the correct answer in the following cell.
</div>

In [ ]:
display_quiz("../quiz_files/InsertSizeQuiz.json")

With paired-end ATAC-seq data we can separate by fragment size to obtain Transposase HyperSensitive Sites (THSS) and Nucleosomal Fragments. Alternatively, some choose to keep the data together as a more general measure of "accessible" sites. These THSS correspond to smaller fragments than the nucleosomes and correspond to transcription factor binding sites.

<img src="../images/Tutorial1/LessonImages/MethodAnimation.gif" alt="Drawing" style="width: 400px;"/>

We'll show you how to separate the small and large fragments into different bam files.

In [ ]:
# Filter by insert size:
! samtools view -h Tutorial2/InputFiles/CTL_dedup.bam | awk 'substr($0,1,1)=="@" || ($9>= 150 && $9<=250) || ($9<=-150 && $9>=-250)' | samtools view -b > Tutorial2/InputFiles/CTL_Nucleosomal.bam
! samtools view -h Tutorial2/InputFiles/CTL_dedup.bam | awk 'substr($0,1,1)=="@" || ($9>= 10 && $9<=125) || ($9<=-10 && $9>=-125)' | samtools view -b > Tutorial2/InputFiles/CTL_THSS.bam
# Do the same for the mutant:
! samtools view -h Tutorial2/InputFiles/Mutant_dedup.bam | awk 'substr($0,1,1)=="@" || ($9>= 150 && $9<=250) || ($9<=-150 && $9>=-250)' | samtools view -b > Tutorial2/InputFiles/Mutant_Nucleosomal.bam
! samtools view -h Tutorial2/InputFiles/Mutant_dedup.bam | awk 'substr($0,1,1)=="@" || ($9>= 10 && $9<=125) || ($9<=-10 && $9>=-125)' | samtools view -b > Tutorial2/InputFiles/Mutant_THSS.bam

For the rest of this tutorial, we'll use the bam files that contain all the reads as many use this as a general measurement of "accessibility". However, you can use these split bam files to create bigwigs, view them in a genome browser, and create average profiles around features as demonstrated earlier. You can also use them in our downstream analysis in lieu of the combined file that we will show in our examples.  

<div class="alert-info" style="font-size:200%">
STEP 3: Peak Detection
</div>
<img src="../images/Tutorial2/LessonImages/Peak.jpg" alt="Drawing" style="width: 200px;"/>

Accessible sites are loci with a pileup of reads in "Peaks". 

### Opitional Note:
Tn5 insertion of adapters leaves a 9 bp gap. In the end, this probably won't impact the results much. However, to be safe we can shift the reads to account for this insertion offset.

<img src="../images/Tutorial2/LessonImages/adapterinsert9bp.jpg" alt="Drawing" style="width: 300px;"/>

Image adjusted from: [Grandi et al., Nature Protocols 2022](https://www.nature.com/articles/s41596-022-00692-9)

The alignmentSieve command from [deeptools](https://anaconda.org/bioconda/deeptools) allows us to shift the reads accordingly.

In [ ]:
! alignmentSieve -p $numthreadsint --ATACshift -b Tutorial2/InputFiles/CTL_dedup.bam -o Tutorial2/InputFiles/CTL_shift.bam
! alignmentSieve -p $numthreadsint --ATACshift -b Tutorial2/InputFiles/Mutant_dedup.bam -o Tutorial2/InputFiles/Mutant_shift.bam

Let's identify peaks genome-wide using [macs2](https://pypi.org/project/MACS2/).

In [ ]:
# If your data is single-end (not paired-end), use -f BAM instead.
! macs2 callpeak -f BAMPE -g hs --keep-dup all --cutoff-analysis -n CTL -t Tutorial2/InputFiles/CTL_shift.bam --outdir Tutorial2/Peaks/ 2> Tutorial2/Peaks/macs2_CTL.log
! macs2 callpeak -f BAMPE -g hs --keep-dup all --cutoff-analysis -n Mutant -t Tutorial2/InputFiles/Mutant_shift.bam --outdir Tutorial2/Peaks/ 2> Tutorial2/Peaks/macs2_Mutant.log

macs2 provides a .narrowPeak file specifying the coordinates of the peaks, an .xls file with additional information, and a .bed file with the summits of the peaks.

Let's view the first 10 lines of the .narrowPeak file.

In [ ]:
# If you have any trouble with this command, try uninstalling macs2 and reinstalling with pip
! head Tutorial2/Peaks/CTL_peaks.narrowPeak

We can also visually inspect the peaks compared to the signal in IGV:

In [ ]:
igv_notebook.init()

In [ ]:
igv_notebook.init()
myigv = igv_notebook.Browser(
    {
        "genome": "hg38",
        "locus": "chr4:55,570,000-55,670,000"
    }
)
myigv.load_track(
{
        "name": "CTL",
        "url": "Tutorial2/BigWigFiles/Control.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "CTL_peaks",
        "url": "Tutorial2/Peaks/CTL_peaks.narrowPeak",
        "format": "bed",
        "type": "annotation"
    }
    
)
myigv.load_track(
{
        "name": "Mutant",
        "url": "Tutorial2/BigWigFiles/Mutant.bw",
        "format": "bigwig",
        "type": "wig"
    }
    
)
myigv.load_track(
{
        "name": "Mutant_peaks",
        "url": "Tutorial2/Peaks/Mutant_peaks.narrowPeak",
        "format": "bed",
        "type": "annotation"
    }
    
)

## Conclusion

This Jupyter Notebook provided a comprehensive guide to visualizing and identifying peaks in ATAC-seq data.  We successfully set up the necessary computational environment, organized our data, and generated bigwig files for visualization in IGV.  The notebook demonstrated the creation of average read density profiles around transcription start sites using deepTools, providing insights into genomic signal distribution.  Furthermore, we performed peak calling using macs2, identifying regions of open chromatin, and visualized these peaks alongside the signal in IGV.  The analysis also included an examination of insert size distributions to differentiate between nucleosomal fragments and transposase hypersensitive sites.  Optional steps, such as filtering reads by insert size and accounting for Tn5 insertion offset, were also presented.  This module successfully covered key steps in ATAC-seq data analysis, laying the groundwork for more advanced downstream analyses detailed in the subsequent notebook.

## Clean Up
We have completed the first downstream processing steps and are ready to move on to some additional downstream analysis. Take a break here or move on to the next tutorial.

[Downtream Analysis](./ATACseq_Tutorial3_Downstream.ipynb)

<div class="alert alert-block alert-danger">
    <b>&#128721; Caution:</b> Remember to shut down your VM after you are finished with your work in order to avoid incurring additional charges.
</div>